In [ ]:

import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import random



# # Configuration constants
# URL = "https://www.randhawa.us/games/retailer/nyu.html"
# TARGET_TOTAL = 2000  # Total inventory
# WEEKS = 15           # Game duration in weeks
# SIMULATIONS = 2   # Number of simulation rounds to run
# WAIT_TIMEOUT = 2    # Maximum seconds to wait for elements

# # Button IDs/text for locating elements (adjust as needed based on actual page)
# BUTTONS = {
#     'restart': "practiceButton",       # Restart Game
#     'maintain': "maintainButton",      # Maintain Price
#     'md10': "tenButton",               # 10% Markdown
#     'md20': "twentyButton",            # 20% Markdown
#     'md40': "fortyButton"              # 40% Markdown
# }


# Implementation

# tRY 2

In [ ]:
# #!/usr/bin/env python3
# """
# Retailer Game Automation Script
# ----------------------------
# This script automates playing the Retailer Game 1,000 times using Selenium.
# It tracks detailed weekly results and final outcomes in separate CSV files.
# """

# import csv
# import os
# import time
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import TimeoutException, NoSuchElementException

# # Game URL
# GAME_URL = "https://www.randhawa.us/games/retailer/nyu.html"

# # Button IDs for various actions
# BUTTONS = {
#     'restart': "practiceButton",  # Restart Game
#     'maintain': "maintainButton", # Maintain Price
#     'md10': "tenButton",          # 10% Markdown
#     'md20': "twentyButton",       # 20% Markdown
#     'md40': "fortyButton"         # 40% Markdown
# }

# def creeping_strategy():
#     """
#     Generator function that implements a price markdown strategy.
    
#     This strategy uses an adaptive approach based on the sales pace
#     and remaining weeks to try to clear inventory by the end of the game.
    
#     Yields:
#         str: A key from BUTTONS dictionary indicating the next action
#     """
#     week = 1
#     last_action = None
    
#     # First action (needs to be primed)
#     action = yield None
    
#     while week <= 14:
#         # Receive cumulative sales from the main loop
#         cum_sales = yield action
        
#         # Calculate the remaining inventory
#         remaining_inventory = 2000 - cum_sales
        
#         # Calculate weeks left
#         weeks_left = 15 - week
        
#         if weeks_left == 0:
#             # No more decisions to make
#             action = 'maintain'
#             continue
            
#         # Calculate the sales rate needed to clear inventory
#         needed_weekly_sales_rate = remaining_inventory / weeks_left if weeks_left > 0 else float('inf')
        
#         # Calculate the current weekly sales rate (average of past weeks)
#         current_weekly_sales_rate = cum_sales / week if week > 0 else 0
        
#         # Calculate the ratio of current sales rate to needed rate
#         sales_rate_ratio = current_weekly_sales_rate / needed_weekly_sales_rate if needed_weekly_sales_rate > 0 else float('inf')
        
#         # Logic for making decisions
#         if sales_rate_ratio >= 1.2:
#             # Sales are well ahead of pace - maintain price
#             action = 'maintain'
#         elif sales_rate_ratio >= 1.0:
#             # Sales are on pace - maintain or small markdown
#             action = 'maintain' if week < 8 else 'md10'
#         elif sales_rate_ratio >= 0.8:
#             # Slightly behind pace - 10% markdown
#             action = 'md10'
#         elif sales_rate_ratio >= 0.6:
#             # Moderately behind - 20% markdown
#             action = 'md20'
#         else:
#             # Significantly behind - 40% markdown
#             action = 'md40'
        
#         # Apply price adjustment restrictions (once price is reduced, cannot go back up)
#         if last_action == 'md10' and action == 'maintain':
#             action = 'md10'
#         elif last_action in ['md20', 'md40'] and action in ['maintain', 'md10']:
#             action = last_action
#         elif last_action == 'md40':
#             action = 'md40'  # Once at 40%, stay there
        
#         last_action = action
#         week += 1


# def play_simulation(driver, sim_num, week_writer, outcome_writer, strategy_generator):
#     """
#     Play one complete simulation of the Retailer Game
    
#     Args:
#         driver: Selenium WebDriver instance
#         sim_num: Simulation number (for tracking)
#         week_writer: CSV writer for weekly detailed data
#         outcome_writer: CSV writer for final outcomes
#         strategy_generator: Generator function that yields next action
#     """
#     try:
#         # Navigate to the game URL if not already there
#         if driver.current_url != GAME_URL:
#             driver.get(GAME_URL)
#             time.sleep(1)  # Brief pause for page load
        
#         # Click the Restart Game button
#         restart_button = WebDriverWait(driver, 10).until(
#             EC.element_to_be_clickable((By.ID, BUTTONS['restart']))
#         )
#         restart_button.click()
        
#         # Initialize tracking variables
#         week = 1
#         inventory = 2000
#         disallowed_actions = set()
#         total_revenue = 0.0
        
#         # Prime the strategy generator
#         strategy = strategy_generator()
#         next(strategy)  # Prime the generator
#         action_key = strategy.send(None)  # Get first action
        
#         # Main game loop
#         while week <= 15 and inventory > 0:
#             # Wait for the table to load/update
#             table = WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.ID, "result-table"))
#             )
            
#             # Extract data from the table - all rows are in the thead section
#             rows = table.find_elements(By.XPATH, ".//thead/tr")
            
#             # Skip the header row (first row)
#             if len(rows) < 2:  # No data rows yet
#                 current_week = 0
#                 current_price = 60.0  # Initial price (based on provided example)
#                 current_sales = 0
#                 current_inventory = 2000
#             else:
#                 last_row = rows[-1]  # Last row has the current week's data
#                 cells = last_row.find_elements(By.TAG_NAME, "td")
                
#                 # Parse the data from the cells
#                 current_week = int(cells[0].text) if cells[0].text else 0
#                 current_price = float(cells[1].text) if cells[1].text else 0.0
#                 current_sales = int(cells[2].text) if cells[2].text else 0
#                 current_inventory = int(cells[3].text) if cells[3].text else 0
            
#             # Update tracking variables
#             inventory = current_inventory
#             total_revenue += current_price * current_sales
            
#             # Write week details to CSV
#             week_writer.writerow([
#                 sim_num, 
#                 current_week, 
#                 current_price, 
#                 current_sales, 
#                 current_inventory
#             ])
            
#             # Check if the game is over
#             if week == 15 or inventory == 0:
#                 break
            
#             # Get next action from strategy generator
#             cumulative_sales = 2000 - inventory
#             action_key = strategy.send(cumulative_sales)
            
#             # Apply the action restrictions
#             if 'maintain' in disallowed_actions and action_key == 'maintain':
#                 action_key = 'md10'  # Default to 10% if maintain is disallowed
#             if 'md10' in disallowed_actions and action_key == 'md10':
#                 action_key = 'md20'  # Default to 20% if 10% is disallowed
            
#             # Click the button for the chosen action
#             if action_key in BUTTONS and action_key not in disallowed_actions:
#                 try:
#                     button = WebDriverWait(driver, 5).until(
#                         EC.element_to_be_clickable((By.ID, BUTTONS[action_key]))
#                     )
#                     button.click()
                    
#                     # Update disallowed actions based on current action
#                     if action_key == 'md10':
#                         disallowed_actions.add('maintain')
#                     elif action_key == 'md20':
#                         disallowed_actions.add('maintain')
#                         disallowed_actions.add('md10')
#                     elif action_key == 'md40':
#                         # After 40%, no further actions allowed
#                         disallowed_actions.update(['maintain', 'md10', 'md20', 'md40'])
#                 except (TimeoutException, NoSuchElementException):
#                     print(f"Button {action_key} not clickable in simulation {sim_num}, week {week}")
            
#             # Increment week counter
#             week += 1
            
#             # Small wait to ensure game state updates
#             time.sleep(0.5)
        
#         # Extract final results
#         try:
#             user_revenue = WebDriverWait(driver, 5).until(
#                 EC.presence_of_element_located((By.ID, "rev"))
#             ).text
#             perfect_revenue = driver.find_element(By.ID, "perfect").text
#             percentage_diff = driver.find_element(By.ID, "percentage").text.replace(", Difference: ", "")
            
#             # Clean up the revenue values (remove $ and commas)
#             user_revenue = user_revenue.replace("$", "").replace(",", "")
#             perfect_revenue = perfect_revenue.replace("$", "").replace(",", "")
            
#             # Write outcome to CSV
#             outcome_writer.writerow([
#                 sim_num,
#                 user_revenue,
#                 perfect_revenue,
#                 percentage_diff
#             ])
            
#         except (TimeoutException, NoSuchElementException) as e:
#             print(f"Error extracting final results for simulation {sim_num}: {e}")
#             # Write whatever data we have
#             outcome_writer.writerow([sim_num, total_revenue, "N/A", "N/A"])
    
#     except Exception as e:
#         print(f"Error in simulation {sim_num}: {e}")


# def main():
#     """
#     Main function to run the automation
#     """
#     # Set up Chrome options for headless browser
#     chrome_options = Options()
#     #chrome_options.add_argument("--headless")
#     chrome_options.add_argument("--disable-gpu")
#     chrome_options.add_argument("--window-size=1920x1080")
#     chrome_options.add_argument("--no-sandbox")
    
#     # Initialize WebDriver
#     driver = webdriver.Chrome(options=chrome_options)
    
#     try:
#         # Create output directory if it doesn't exist
#         os.makedirs("results", exist_ok=True)
        
#         # Set up CSV files
#         with open("results/week_detail.csv", 'w', newline='') as week_file, \
#              open("results/outcome.csv", 'w', newline='') as outcome_file:
            
#             # Create CSV writers
#             week_writer = csv.writer(week_file)
#             outcome_writer = csv.writer(outcome_file)
            
#             # Write headers
#             week_writer.writerow([
#                 "Simulation Number", "Week", "Price", "Sales", "Remaining Inventory"
#             ])
#             outcome_writer.writerow([
#                 "Simulation Number", "Your revenue", "Perfect foresight strategy", "Difference (%)"
#             ])
            
#             # Run simulations
#             total_simulations = 2
#             for sim_num in range(1, total_simulations + 1):
#                 print(f"Running simulation {sim_num} of {total_simulations}")
#                 play_simulation(
#                     driver, 
#                     sim_num, 
#                     week_writer, 
#                     outcome_writer,
#                     creeping_strategy
#                 )
                
#                 # Optional: Add a small delay between simulations
#                 time.sleep(0.5)
                
#         print("All simulations complete!")
    
#     finally:
#         # Always close the driver
#         driver.quit()


# if __name__ == "__main__":
#     main()

Running simulation 1 of 2
Running simulation 2 of 2
Button md20 not clickable in simulation 2, week 2
All simulations complete!


# Atttempt 3

## Combo creation

In [2]:
from selenium import webdriver
import pandas as pd
from re import sub
import timeit
import itertools

# Create a list of all possible combos

x = [60, 54, 48, 36]
combos = []

for i in itertools.product(x,repeat=14):
    if i[0] >= i[1] >= i[2] >= i[3] >= i[4] >= i[5] >= i[6]>= i[7] >= i[8]>= i[9] >= i[10] >= i[11] >= i[12] >= i[13]:
        combos.append(i)

check = pd.DataFrame(combos)

combos_list = []

for i in combos:
    combos_list.append(list(i))

# First step in each combo is always 60

for i in combos_list:
    i.insert(0,60)

# Assign an ID to each combo

for i in range(len(combos_list)):
    combos_list[i].insert(0,i)

combos_df = pd.DataFrame(combos_list)

combos_df = combos_df.rename(columns={0:"combo_number"})

print(combos_df)

     combo_number   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15
0               0  60  60  60  60  60  60  60  60  60  60  60  60  60  60  60
1               1  60  60  60  60  60  60  60  60  60  60  60  60  60  60  54
2               2  60  60  60  60  60  60  60  60  60  60  60  60  60  60  48
3               3  60  60  60  60  60  60  60  60  60  60  60  60  60  60  36
4               4  60  60  60  60  60  60  60  60  60  60  60  60  60  54  54
..            ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
675           675  60  48  48  48  48  36  36  36  36  36  36  36  36  36  36
676           676  60  48  48  48  36  36  36  36  36  36  36  36  36  36  36
677           677  60  48  48  36  36  36  36  36  36  36  36  36  36  36  36
678           678  60  48  36  36  36  36  36  36  36  36  36  36  36  36  36
679           679  60  36  36  36  36  36  36  36  36  36  36  36  36  36  36

[680 rows x 16 columns]


## Transform to action

In [5]:
import pandas as pd

def price_to_action(price_df):
    # Define the mapping from price change to action
    def get_action(prev_price, current_price):
        if current_price == prev_price:
            return 'A'  # Maintain price
        elif current_price == 54:
            return 'B'  # 10% markdown
        elif current_price == 48:
            return 'C'  # 20% markdown
        elif current_price == 36:
            return 'D'  # 40% markdown
        else:
            return '?'  # Unknown price

    # Copy the combo_number column
    action_df = pd.DataFrame()
    action_df['combo_number'] = price_df['combo_number']

    # Convert price values to actions
    for col_idx, col in enumerate(price_df.columns[1:], start=1):
        week = col  # Week number as column name
        if col_idx == 1:
            # First week: always maintain list price
            action_df[week] = 'A'
        else:
            prev_week = price_df.columns[col_idx - 1]
            action_df[week] = [
                get_action(prev, curr)
                for prev, curr in zip(price_df[prev_week], price_df[week])
            ]

    return action_df

# Assuming your original DataFrame is called `df`
action_df = price_to_action(combos_df)

# Print or save the transformed DataFrame
print(action_df.head())
action_df.to_csv("action_df.csv", index=False)


   combo_number  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15
0             0  A  A  A  A  A  A  A  A  A  A  A  A  A  A  A
1             1  A  A  A  A  A  A  A  A  A  A  A  A  A  A  B
2             2  A  A  A  A  A  A  A  A  A  A  A  A  A  A  C
3             3  A  A  A  A  A  A  A  A  A  A  A  A  A  A  D
4             4  A  A  A  A  A  A  A  A  A  A  A  A  A  B  A


In [ ]:
display(action_df)

,combo_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A
1,1,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B
2,2,A,A,A,A,A,A,A,A,A,A,A,A,A,A,C
3,3,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D
4,4,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,675,A,C,A,A,A,D,A,A,A,A,A,A,A,A,A
676,676,A,C,A,A,D,A,A,A,A,A,A,A,A,A,A
677,677,A,C,A,D,A,A,A,A,A,A,A,A,A,A,A
678,678,A,C,D,A,A,A,A,A,A,A,A,A,A,A,A


In [2]:
import pandas as pd

action_df = pd.read_csv("action_df.csv")


print(action_df)

     combo_number  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15
0               0  A  A  A  A  A  A  A  A  A  A  A  A  A  A  A
1               1  A  A  A  A  A  A  A  A  A  A  A  A  A  A  B
2               2  A  A  A  A  A  A  A  A  A  A  A  A  A  A  C
3               3  A  A  A  A  A  A  A  A  A  A  A  A  A  A  D
4               4  A  A  A  A  A  A  A  A  A  A  A  A  A  B  A
..            ... .. .. .. .. .. .. .. .. .. .. .. .. .. .. ..
675           675  A  C  A  A  A  D  A  A  A  A  A  A  A  A  A
676           676  A  C  A  A  D  A  A  A  A  A  A  A  A  A  A
677           677  A  C  A  D  A  A  A  A  A  A  A  A  A  A  A
678           678  A  C  D  A  A  A  A  A  A  A  A  A  A  A  A
679           679  A  D  A  A  A  A  A  A  A  A  A  A  A  A  A

[680 rows x 16 columns]


# Simulation Run

### V3

In [12]:
#!/usr/bin/env python3
"""
Retail Markdown Game Simulator
==============================
This script automates playing the Retailer Game using predefined price markdown strategies.
It records results in two CSV files:
- week_detail.csv: Contains weekly data for each simulation
- outcome.csv: Contains final performance metrics for each simulation
"""

import csv
import time
import re
import pandas as pd
import itertools
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import traceback

# Game URL
GAME_URL = "https://www.randhawa.us/games/retailer/nyu.html"

def generate_price_strategies():
    """
    Generate all valid price markdown strategies and returns as a DataFrame.
    
    Returns:
        pandas.DataFrame: DataFrame with all valid price combinations
    """
    # All possible prices
    prices = [60, 54, 48, 36]
    
    # Generate all valid combinations (non-increasing prices)
    valid_combos = []
    combo_id = 0
    
    for combo in itertools.product(prices, repeat=14):
        # Check if prices are non-increasing
        if all(combo[i] >= combo[i+1] for i in range(13)):
            # First week is always 60
            full_combo = [combo_id, 60] + list(combo)
            valid_combos.append(full_combo)
            combo_id += 1
    
    # Create DataFrame
    columns = ["combo_number"] + [str(i) for i in range(1, 16)]
    df = pd.DataFrame(valid_combos, columns=columns)
    
    return df

def convert_to_actions(price_df):
    """
    Convert price values to action letters.
    
    Args:
        price_df: DataFrame with price values
        
    Returns:
        pandas.DataFrame: DataFrame with action letters
    """
    action_df = pd.DataFrame()
    action_df['combo_number'] = price_df['combo_number']
    
    # First week is always maintain (A)
    action_df['1'] = 'A'
    
    # For subsequent weeks, determine the action based on price change
    for week in range(2, 16):
        curr_week = str(week)
        prev_week = str(week - 1)
        
        actions = []
        for _, row in price_df.iterrows():
            prev_price = row[prev_week]
            curr_price = row[curr_week]
            
            if curr_price == prev_price:
                action = 'A'  # Maintain
            elif curr_price == 54:
                action = 'B'  # 10% markdown
            elif curr_price == 48:
                action = 'C'  # 20% markdown
            elif curr_price == 36:
                action = 'D'  # 40% markdown
            else:
                action = '?'  # Unknown (shouldn't happen)
                
            actions.append(action)
            
        action_df[curr_week] = actions
    
    return action_df

def setup_browser():
    """
    Set up and return a configured browser instance.
    
    Returns:
        webdriver.Chrome: Configured Chrome WebDriver
    """
    chrome_options = Options()
    # Uncomment to run headless (no GUI)
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # Initialize the browser
    try:
        # If using webdriver_manager
        from webdriver_manager.chrome import ChromeDriverManager
        browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    except:
        # Fallback to local chromedriver
        browser = webdriver.Chrome(options=chrome_options)
        
    # Navigate to the game and maximize window
    browser.get(GAME_URL)
    browser.maximize_window()
    
    return browser

def run_simulation(browser, combo_row, simulation_number):
    """
    Run a single simulation using the specified pricing strategy.
    
    Args:
        browser: WebDriver instance
        combo_row: DataFrame row with actions for each week
        simulation_number: Identifier for this simulation
    
    Returns:
        tuple: (week_data, outcome_data) containing results
    """
    wait = WebDriverWait(browser, 10)
    
    # Map letters to button IDs
    action_map = {
        'A': 'maintainButton',  # Maintain price
        'B': 'tenButton',       # 10% markdown
        'C': 'twentyButton',    # 20% markdown
        'D': 'fortyButton'      # 40% markdown
    }
    
    # Click the "Start Game" button
    try:
        start_button = wait.until(EC.element_to_be_clickable((By.ID, "practiceButton")))
        start_button.click()
        time.sleep(0.00001)  # Wait for game to initialize
    except Exception as e:
        # If error, refresh and try again
        print(f"Error starting game, refreshing page: {e}")
        browser.refresh()
        time.sleep(0.00002)
        start_button = wait.until(EC.element_to_be_clickable((By.ID, "practiceButton")))
        start_button.click()
        time.sleep(0.0002)
    
    combo_id = int(combo_row['combo_number'])
    week_data = []
    
    # Execute the strategy for weeks 2-15 (week 1 is automatic)
    for week in range(2, 16):
        try:
            week_str = str(week)
            action_letter = combo_row[week_str]
            button_id = action_map[action_letter]
            
            # Wait for and click the appropriate button
            button = wait.until(EC.element_to_be_clickable((By.ID, button_id)))
            button.click()
            #time.sleep(0.000001)  # Allow time for game to update
            
        except TimeoutException:
            #print(f"Button not clickable in simulation {simulation_number}, week {week}")
            # Some actions might be disabled due to previous choices
            continue
        except Exception as e:
            #print(f"Error in week {week}: {e}")
            continue
    
    # Wait for final results
    time.sleep(0.0001)
    
    # Extract results from all weeks
    try:
        # Find the results table
        table = wait.until(EC.presence_of_element_located((By.ID, "result-table")))
        rows = table.find_elements(By.TAG_NAME, "tr")
        
        # Skip header row
        for row in rows[1:]:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 4:
                week_num = int(cells[0].text)
                price = int(cells[1].text)
                sales = int(cells[2].text)
                inventory = int(cells[3].text)
                
                week_data.append([
                    simulation_number,
                    combo_id,
                    week_num,
                    price,
                    sales,
                    inventory
                ])
    except Exception as e:
        print(f"Error extracting week data: {e}")
    
    # Extract final outcome
    try:
        # Find revenue and perfect foresight elements
        revenue_elem = browser.find_element(By.ID, "rev")
        perfect_elem = browser.find_element(By.ID, "perfect")
        
        # Clean the text to get just the numbers
        revenue = int(re.sub(r'[^0-9]', '', revenue_elem.text))
        perfect = int(re.sub(r'[^0-9]', '', perfect_elem.text))
        
        # Extract difference percentage (from percentage element if available)
        try:
            pct_elem = browser.find_element(By.ID, "percentage")
            difference_pct = float(re.sub(r'[^0-9.]', '', pct_elem.text))
        except:
            # Calculate manually if element not found
            difference_pct = round(((perfect - revenue) / perfect) * 100, 2) if perfect > 0 else 0
        
        outcome_data = [
            simulation_number,
            combo_id,
            revenue,
            perfect,
            difference_pct
        ]
    except Exception as e:
        print(f"Error extracting outcome: {e}")
        outcome_data = [simulation_number, combo_id, 0, 0, 0]
    
    return week_data, outcome_data

def main():
    """Main function to run the simulation"""
    # Either generate strategies or use a pre-saved CSV
    try:
        action_df = pd.read_csv('action_df.csv')
        print(f"Loaded {len(action_df)} strategy combinations from action_df.csv")
    except:
        print("Creating price strategies...")
        price_df = generate_price_strategies()
        action_df = convert_to_actions(price_df)
        action_df.to_csv('action_df.csv', index=False)
        print(f"Generated {len(action_df)} strategy combinations")
    
    # Initialize data collectors
    all_week_data = []
    all_outcome_data = []
    
    # Initialize browser
    browser = setup_browser()
    
    # How many simulations to run (set to a small number for testing)
    num_simulations = 680  # Change this to run more or fewer simulations
    
    try:
        # Loop through each combo in the action_df (limited by num_simulations)
        for index, combo_row in action_df.head(num_simulations).iterrows():
            simulation_number = index + 1
            print(f"Running simulation {simulation_number}/{num_simulations}: Combo {combo_row['combo_number']}")
            
            try:
                # Run the simulation and collect results
                week_data, outcome_data = run_simulation(browser, combo_row, simulation_number)
                
                # Store results
                all_week_data.extend(week_data)
                all_outcome_data.append(outcome_data)
                
                # Save results every 10 simulations
                if simulation_number % 10 == 0:
                    print(f"Saving interim results after {simulation_number} simulations...")
                    
                    # Save week detail data
                    week_df = pd.DataFrame(
                        all_week_data, 
                        columns=["Simulation Number", "comboID", "Week", "Price", "Sales", "Remaining Inventory"]
                    )
                    week_df.to_csv(f"week_detail_interim_{simulation_number}.csv", index=False)
                    
                    # Save outcome data
                    outcome_df = pd.DataFrame(
                        all_outcome_data, 
                        columns=["Simulation Number", "comboID", "Your revenue", "Perfect foresight strategy", "Difference (%)"]
                    )
                    outcome_df.to_csv(f"outcome_interim_{simulation_number}.csv", index=False)
                
            except Exception as e:
                print(f"Error in simulation {simulation_number}: {e}")
                traceback.print_exc()
                continue
    
    except Exception as e:
        print(f"Fatal error: {e}")
        traceback.print_exc()
    
    finally:
        # Always close the browser when done
        browser.quit()
        
        # Save final results
        if all_week_data:
            week_df = pd.DataFrame(
                all_week_data, 
                columns=["Simulation Number", "comboID", "Week", "Price", "Sales", "Remaining Inventory"]
            )
            week_df.to_csv("week_detail.csv", index=False)
        
        if all_outcome_data:
            outcome_df = pd.DataFrame(
                all_outcome_data, 
                columns=["Simulation Number", "comboID", "Your revenue", "Perfect foresight strategy", "Difference (%)"]
            )
            outcome_df.to_csv("outcome.csv", index=False)
        
        print("Simulation completed. Results saved to week_detail.csv and outcome.csv")

if __name__ == "__main__":
    main() 

Loaded 680 strategy combinations from action_df.csv
Running simulation 1/680: Combo 0
Running simulation 2/680: Combo 1
Running simulation 3/680: Combo 2
Running simulation 4/680: Combo 3
Running simulation 5/680: Combo 4
Running simulation 6/680: Combo 5
Running simulation 7/680: Combo 6
Running simulation 8/680: Combo 7
Running simulation 9/680: Combo 8
Running simulation 10/680: Combo 9
Saving interim results after 10 simulations...
Running simulation 11/680: Combo 10
Running simulation 12/680: Combo 11
Running simulation 13/680: Combo 12
Running simulation 14/680: Combo 13
Running simulation 15/680: Combo 14
Running simulation 16/680: Combo 15
Running simulation 17/680: Combo 16
Running simulation 18/680: Combo 17
Running simulation 19/680: Combo 18
Running simulation 20/680: Combo 19
Saving interim results after 20 simulations...
Running simulation 21/680: Combo 20
Running simulation 22/680: Combo 21
Running simulation 23/680: Combo 22
Running simulation 24/680: Combo 23
Running s

KeyboardInterrupt: 

In [14]:
#!/usr/bin/env python3
"""
Retail Markdown Game Simulator
==============================
This script automates playing the Retailer Game using predefined price markdown strategies.
It records results in two CSV files:
- week_detail.csv: Contains weekly data for each simulation
- outcome.csv: Contains final performance metrics for each simulation
"""

import csv
import time
import re
import pandas as pd
import itertools
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import traceback
import os

# Game URL
GAME_URL = "https://www.randhawa.us/games/retailer/nyu.html"

def generate_price_strategies():
    """
    Generate all valid price markdown strategies and returns as a DataFrame.
    
    Returns:
        pandas.DataFrame: DataFrame with all valid price combinations
    """
    # All possible prices
    prices = [60, 54, 48, 36]
    
    # Generate all valid combinations (non-increasing prices)
    valid_combos = []
    combo_id = 0
    
    for combo in itertools.product(prices, repeat=14):
        # Check if prices are non-increasing
        if all(combo[i] >= combo[i+1] for i in range(13)):
            # First week is always 60
            full_combo = [combo_id, 60] + list(combo)
            valid_combos.append(full_combo)
            combo_id += 1
    
    # Create DataFrame
    columns = ["combo_number"] + [str(i) for i in range(1, 16)]
    df = pd.DataFrame(valid_combos, columns=columns)
    
    return df

def convert_to_actions(price_df):
    """
    Convert price values to action letters.
    
    Args:
        price_df: DataFrame with price values
        
    Returns:
        pandas.DataFrame: DataFrame with action letters
    """
    action_df = pd.DataFrame()
    action_df['combo_number'] = price_df['combo_number']
    
    # First week is always maintain (A)
    action_df['1'] = 'A'
    
    # For subsequent weeks, determine the action based on price change
    for week in range(2, 16):
        curr_week = str(week)
        prev_week = str(week - 1)
        
        actions = []
        for _, row in price_df.iterrows():
            prev_price = row[prev_week]
            curr_price = row[curr_week]
            
            if curr_price == prev_price:
                action = 'A'  # Maintain
            elif curr_price == 54:
                action = 'B'  # 10% markdown
            elif curr_price == 48:
                action = 'C'  # 20% markdown
            elif curr_price == 36:
                action = 'D'  # 40% markdown
            else:
                action = '?'  # Unknown (shouldn't happen)
                
            actions.append(action)
            
        action_df[curr_week] = actions
    
    return action_df

def setup_browser():
    """
    Set up and return a configured browser instance.
    
    Returns:
        webdriver.Chrome: Configured Chrome WebDriver
    """
    chrome_options = Options()
    # Uncomment to run headless (no GUI)
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # Initialize the browser
    try:
        # If using webdriver_manager
        from webdriver_manager.chrome import ChromeDriverManager
        browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    except:
        # Fallback to local chromedriver
        browser = webdriver.Chrome(options=chrome_options)
        
    # Navigate to the game and maximize window
    browser.get(GAME_URL)
    browser.maximize_window()
    
    return browser

def run_simulation(browser, combo_row, simulation_number):
    """
    Run a single simulation using the specified pricing strategy.
    
    Args:
        browser: WebDriver instance
        combo_row: DataFrame row with actions for each week
        simulation_number: Identifier for this simulation
    
    Returns:
        tuple: (week_data, outcome_data) containing results
    """
    wait = WebDriverWait(browser, 2)
    
    # Map letters to button IDs
    action_map = {
        'A': 'maintainButton',  # Maintain price
        'B': 'tenButton',       # 10% markdown
        'C': 'twentyButton',    # 20% markdown
        'D': 'fortyButton'      # 40% markdown
    }
    
    # Click the "Start Game" button
    try:
        start_button = wait.until(EC.element_to_be_clickable((By.ID, "practiceButton")))
        start_button.click()
        time.sleep(0.00001)  # Wait for game to initialize
    except Exception as e:
        # If error, refresh and try again
        print(f"Error starting game, refreshing page: {e}")
        browser.refresh()
        #time.sleep(0.00002)
        wait.until(EC.presence_of_element_located((By.ID, "practiceButton")))
        start_button = wait.until(EC.element_to_be_clickable((By.ID, "practiceButton")))
        start_button.click()
        time.sleep(0.0002)
    
    combo_id = int(combo_row['combo_number'])
    week_data = []
    
    # Execute the strategy for weeks 2-15 (week 1 is automatic)
    for week in range(2, 16):
        try:
            week_str = str(week)
            action_letter = combo_row[week_str]
            button_id = action_map[action_letter]
            
            # Wait for and click the appropriate button
            button = wait.until(EC.element_to_be_clickable((By.ID, button_id)))
            button.click()
            #time.sleep(0.000001)  # Allow time for game to update
            
        except TimeoutException:
            #print(f"Button not clickable in simulation {simulation_number}, week {week}")
            # Some actions might be disabled due to previous choices
            continue
        except Exception as e:
            #print(f"Error in week {week}: {e}")
            continue
    
    # Wait for final results
    #time.sleep(0.0001)
    wait.until(EC.presence_of_element_located((By.ID, "rev")))
    
    # Extract results from all weeks
    try:
        # Find the results table
        table = wait.until(EC.presence_of_element_located((By.ID, "result-table")))
        rows = table.find_elements(By.TAG_NAME, "tr")
        
        # Skip header row
        for row in rows[1:]:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 4:
                week_num = int(cells[0].text)
                price = int(cells[1].text)
                sales = int(cells[2].text)
                inventory = int(cells[3].text)
                
                week_data.append([
                    simulation_number,
                    combo_id,
                    week_num,
                    price,
                    sales,
                    inventory
                ])
    except Exception as e:
        print(f"Error extracting week data: {e}")
    
    # Extract final outcome
    try:
        # Find revenue and perfect foresight elements
        revenue_elem = browser.find_element(By.ID, "rev")
        perfect_elem = browser.find_element(By.ID, "perfect")
        
        # Clean the text to get just the numbers
        revenue = int(re.sub(r'[^0-9]', '', revenue_elem.text))
        perfect = int(re.sub(r'[^0-9]', '', perfect_elem.text))
        
        # Extract difference percentage (from percentage element if available)
        try:
            pct_elem = browser.find_element(By.ID, "percentage")
            difference_pct = float(re.sub(r'[^0-9.]', '', pct_elem.text))
        except:
            # Calculate manually if element not found
            difference_pct = round(((perfect - revenue) / perfect) * 100, 2) if perfect > 0 else 0
        
        outcome_data = [
            simulation_number,
            combo_id,
            revenue,
            perfect,
            difference_pct
        ]
    except Exception as e:
        print(f"Error extracting outcome: {e}")
        outcome_data = [simulation_number, combo_id, 0, 0, 0]
    
    return week_data, outcome_data

def main():
    """Main function to run the simulation"""
    # Either generate strategies or use a pre-saved CSV
    try:
        action_df = pd.read_csv('action_df.csv')
        print(f"Loaded {len(action_df)} strategy combinations from action_df.csv")
    except:
        print("Creating price strategies...")
        price_df = generate_price_strategies()
        action_df = convert_to_actions(price_df)
        action_df.to_csv('action_df.csv', index=False)
        print(f"Generated {len(action_df)} strategy combinations")
    
    # Initialize data collectors
    all_week_data = []
    all_outcome_data = []
    
    # Initialize browser
    browser = setup_browser()
    
    # How many simulations to run (set to a small number for testing)
    num_simulations = 680  # Change this to run more or fewer simulations
    
    try:
        # Loop through each combo in the action_df (limited by num_simulations)
        for index, combo_row in action_df.head(num_simulations).iterrows():
            simulation_number = index + 1
            print(f"Running simulation {simulation_number}/{num_simulations}: Combo {combo_row['combo_number']}")
            
            try:
                # Run the simulation and collect results
                week_data, outcome_data = run_simulation(browser, combo_row, simulation_number)
                
                # Store results
                all_week_data.extend(week_data)
                all_outcome_data.append(outcome_data)
                
                # Save results every 100 simulations
                if simulation_number % 100 == 0:
                    print(f"Saving interim results after {simulation_number} simulations...")
                    
                    # Save week detail data
                    week_df = pd.DataFrame(
                        all_week_data, 
                        columns=["Simulation Number", "comboID", "Week", "Price", "Sales", "Remaining Inventory"]
                    )
                    week_df.to_csv(f"week_detail_interim_{simulation_number}.csv", index=False)
                    
                    # Save outcome data
                    outcome_df = pd.DataFrame(
                        all_outcome_data, 
                        columns=["Simulation Number", "comboID", "Your revenue", "Perfect foresight strategy", "Difference (%)"]
                    )
                    outcome_df.to_csv(f"outcome_interim_{simulation_number}.csv", index=False)
                
            except Exception as e:
                print(f"Error in simulation {simulation_number}: {e}")
                traceback.print_exc()
                continue
    
    except Exception as e:
        print(f"Fatal error: {e}")
        traceback.print_exc()
    
    finally:
        # Always close the browser when done
        browser.quit()
        
        # Save final results
        if all_week_data:
            week_df = pd.DataFrame(
                all_week_data, 
                columns=["Simulation Number", "comboID", "Week", "Price", "Sales", "Remaining Inventory"]
            )
            #week_df.to_csv("week_detail.csv", index=False)
            write_header = not os.path.exists("week_detail.csv")
            week_df.to_csv("week_detail.csv", mode='a', index=False, header=write_header)
        
        if all_outcome_data:
            outcome_df = pd.DataFrame(
                all_outcome_data, 
                columns=["Simulation Number", "comboID", "Your revenue", "Perfect foresight strategy", "Difference (%)"]
            )
            #outcome_df.to_csv("outcome.csv", index=False)
            write_header = not os.path.exists("outcome.csv")
            outcome_df.to_csv("outcome.csv", mode='a', index=False, header=write_header)
        
        print("Simulation completed. Results saved to week_detail.csv and outcome.csv")

if __name__ == "__main__":
    main() 

Loaded 680 strategy combinations from action_df.csv
Running simulation 1/680: Combo 0
Running simulation 2/680: Combo 1
Running simulation 3/680: Combo 2
Running simulation 4/680: Combo 3
Running simulation 5/680: Combo 4
Running simulation 6/680: Combo 5
Running simulation 7/680: Combo 6
Running simulation 8/680: Combo 7
Running simulation 9/680: Combo 8
Running simulation 10/680: Combo 9
Running simulation 11/680: Combo 10
Running simulation 12/680: Combo 11
Running simulation 13/680: Combo 12
Running simulation 14/680: Combo 13
Running simulation 15/680: Combo 14
Running simulation 16/680: Combo 15
Running simulation 17/680: Combo 16
Running simulation 18/680: Combo 17
Running simulation 19/680: Combo 18
Running simulation 20/680: Combo 19
Running simulation 21/680: Combo 20
Running simulation 22/680: Combo 21
Running simulation 23/680: Combo 22
Running simulation 24/680: Combo 23
Running simulation 25/680: Combo 24
Running simulation 26/680: Combo 25
Simulation completed. Results s

KeyboardInterrupt: 

# FINAL

In [1]:
#!/usr/bin/env python3
"""
Retail Markdown Game Simulator
==============================
This script automates playing the Retailer Game using predefined price markdown strategies.
It records results in two CSV files:
- week_detail.csv: Contains weekly data for each simulation
- outcome.csv: Contains final performance metrics for each simulation
"""

import csv
import time
import re
import pandas as pd
import itertools
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import traceback
import os

# Game URL
GAME_URL = "https://www.randhawa.us/games/retailer/nyu.html"

def generate_price_strategies():
    """
    Generate all valid price markdown strategies and returns as a DataFrame.
    
    Returns:
        pandas.DataFrame: DataFrame with all valid price combinations
    """
    # All possible prices
    prices = [60, 54, 48, 36]
    
    # Generate all valid combinations (non-increasing prices)
    valid_combos = []
    combo_id = 0
    
    for combo in itertools.product(prices, repeat=14):
        # Check if prices are non-increasing
        if all(combo[i] >= combo[i+1] for i in range(13)):
            # First week is always 60
            full_combo = [combo_id, 60] + list(combo)
            valid_combos.append(full_combo)
            combo_id += 1
    
    # Create DataFrame
    columns = ["combo_number"] + [str(i) for i in range(1, 16)]
    df = pd.DataFrame(valid_combos, columns=columns)
    
    return df

def convert_to_actions(price_df):
    """
    Convert price values to action letters.
    
    Args:
        price_df: DataFrame with price values
        
    Returns:
        pandas.DataFrame: DataFrame with action letters
    """
    action_df = pd.DataFrame()
    action_df['combo_number'] = price_df['combo_number']
    
    # First week is always maintain (A)
    action_df['1'] = 'A'
    
    # For subsequent weeks, determine the action based on price change
    for week in range(2, 16):
        curr_week = str(week)
        prev_week = str(week - 1)
        
        actions = []
        for _, row in price_df.iterrows():
            prev_price = row[prev_week]
            curr_price = row[curr_week]
            
            if curr_price == prev_price:
                action = 'A'  # Maintain
            elif curr_price == 54:
                action = 'B'  # 10% markdown
            elif curr_price == 48:
                action = 'C'  # 20% markdown
            elif curr_price == 36:
                action = 'D'  # 40% markdown
            else:
                action = '?'  # Unknown (shouldn't happen)
                
            actions.append(action)
            
        action_df[curr_week] = actions
    
    return action_df

def setup_browser():
    """
    Set up and return a configured browser instance.
    
    Returns:
        webdriver.Chrome: Configured Chrome WebDriver
    """
    chrome_options = Options()
    # Uncomment to run headless (no GUI)
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    # Performance optimizations
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-images")
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")
    chrome_options.page_load_strategy = 'eager'
    
    # Initialize the browser
    try:
        # If using webdriver_manager
        from webdriver_manager.chrome import ChromeDriverManager
        browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    except:
        # Fallback to local chromedriver
        browser = webdriver.Chrome(options=chrome_options)
    
    # Set shorter timeouts
    browser.set_page_load_timeout(5)
    browser.set_script_timeout(5)
        
    # Navigate to the game and maximize window
    browser.get(GAME_URL)
    browser.maximize_window()
    
    return browser

def run_simulation(browser, combo_row, simulation_number):
    """
    Run a single simulation using the specified pricing strategy.
    
    Args:
        browser: WebDriver instance
        combo_row: DataFrame row with actions for each week
        simulation_number: Identifier for this simulation
    
    Returns:
        tuple: (week_data, outcome_data) containing results
    """
    wait = WebDriverWait(browser, 2)
    
    # Map letters to button IDs
    action_map = {
        'A': 'maintainButton',  # Maintain price
        'B': 'tenButton',       # 10% markdown
        'C': 'twentyButton',    # 20% markdown
        'D': 'fortyButton'      # 40% markdown
    }
    
    # Click the "Start Game" button
    try:
        start_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#practiceButton")))
        start_button.click()
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#result-table tr:nth-child(2)")))
    except Exception as e:
        # If error, refresh and try again
        print(f"Error starting game, refreshing page: {e}")
        browser.refresh()
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#practiceButton")))
        start_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#practiceButton")))
        start_button.click()
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#result-table tr:nth-child(2)")))
    
    combo_id = int(combo_row['combo_number'])
    week_data = []
    
    # Execute the strategy for weeks 2-15 (week 1 is automatic)
    for week in range(2, 16):
        try:
            week_str = str(week)
            action_letter = combo_row[week_str]
            button_id = action_map[action_letter]
            
            # Wait for and click the appropriate button using CSS selector
            button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#{button_id}")))
            button.click()
            
        except TimeoutException:
            # Some actions might be disabled due to previous choices
            continue
        except Exception as e:
            continue
    
    # Wait for final results
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#rev")))
    
    # Extract results from all weeks
    try:
        # Find the results table
        table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#result-table")))
        rows = table.find_elements(By.TAG_NAME, "tr")
        
        # Skip header row
        for row in rows[1:]:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 4:
                week_num = int(cells[0].text)
                price = int(cells[1].text)
                sales = int(cells[2].text)
                inventory = int(cells[3].text)
                
                week_data.append([
                    simulation_number,
                    combo_id,
                    week_num,
                    price,
                    sales,
                    inventory
                ])
    except Exception as e:
        print(f"Error extracting week data: {e}")
    
    # Extract final outcome
    try:
        # Find revenue and perfect foresight elements using CSS selectors
        revenue_elem = browser.find_element(By.CSS_SELECTOR, "#rev")
        perfect_elem = browser.find_element(By.CSS_SELECTOR, "#perfect")
        
        # Clean the text to get just the numbers
        revenue = int(re.sub(r'[^0-9]', '', revenue_elem.text))
        perfect = int(re.sub(r'[^0-9]', '', perfect_elem.text))
        
        # Extract difference percentage (from percentage element if available)
        try:
            pct_elem = browser.find_element(By.CSS_SELECTOR, "#percentage")
            difference_pct = float(re.sub(r'[^0-9.]', '', pct_elem.text))
        except:
            # Calculate manually if element not found
            difference_pct = round(((perfect - revenue) / perfect) * 100, 2) if perfect > 0 else 0
        
        outcome_data = [
            simulation_number,
            combo_id,
            revenue,
            perfect,
            difference_pct
        ]
    except Exception as e:
        print(f"Error extracting outcome: {e}")
        outcome_data = [simulation_number, combo_id, 0, 0, 0]
    
    return week_data, outcome_data

def main():
    """Main function to run the simulation"""
    # Either generate strategies or use a pre-saved CSV
    try:
        action_df = pd.read_csv('action_df.csv')
        print(f"Loaded {len(action_df)} strategy combinations from action_df.csv")
    except:
        print("Creating price strategies...")
        price_df = generate_price_strategies()
        action_df = convert_to_actions(price_df)
        action_df.to_csv('action_df.csv', index=False)
        print(f"Generated {len(action_df)} strategy combinations")
    
    # Initialize data collectors
    all_week_data = []
    all_outcome_data = []
    
    # Initialize browser
    browser = setup_browser()
    
    # How many simulations to run
    num_simulations = 680
    # Restart browser every X simulations
    browser_restart_frequency = 50
    
    try:
        # Loop through each combo in the action_df (limited by num_simulations)
        for index, combo_row in action_df.head(num_simulations).iterrows():
            simulation_number = index + 1
            print(f"Running simulation {simulation_number}/{num_simulations}: Combo {combo_row['combo_number']}")
            
            # Reset browser every X simulations to prevent memory issues
            if simulation_number % browser_restart_frequency == 0 and simulation_number > 0:
                print(f"Restarting browser after {simulation_number} simulations...")
                browser.quit()
                browser = setup_browser()
            
            try:
                # Run the simulation and collect results
                week_data, outcome_data = run_simulation(browser, combo_row, simulation_number)
                
                # Store results
                all_week_data.extend(week_data)
                all_outcome_data.append(outcome_data)
                
                # Save results every 200 simulations
                if simulation_number % 200 == 0:
                    print(f"Saving interim results after {simulation_number} simulations...")
                    
                    # Save week detail data
                    week_df = pd.DataFrame(
                        all_week_data, 
                        columns=["Simulation Number", "comboID", "Week", "Price", "Sales", "Remaining Inventory"]
                    )
                    week_df.to_csv(f"week_detail_interim_{simulation_number}.csv", index=False)
                    
                    # Save outcome data
                    outcome_df = pd.DataFrame(
                        all_outcome_data, 
                        columns=["Simulation Number", "comboID", "Your revenue", "Perfect foresight strategy", "Difference (%)"]
                    )
                    outcome_df.to_csv(f"outcome_interim_{simulation_number}.csv", index=False)
                
            except Exception as e:
                print(f"Error in simulation {simulation_number}: {e}")
                traceback.print_exc()
                continue
    
    except Exception as e:
        print(f"Fatal error: {e}")
        traceback.print_exc()
    
    finally:
        # Always close the browser when done
        browser.quit()
        
        # Save final results
        if all_week_data:
            week_df = pd.DataFrame(
                all_week_data, 
                columns=["Simulation Number", "comboID", "Week", "Price", "Sales", "Remaining Inventory"]
            )
            write_header = not os.path.exists("week_detail.csv")
            week_df.to_csv("week_detail.csv", mode='a', index=False, header=write_header)
        
        if all_outcome_data:
            outcome_df = pd.DataFrame(
                all_outcome_data, 
                columns=["Simulation Number", "comboID", "Your revenue", "Perfect foresight strategy", "Difference (%)"]
            )
            write_header = not os.path.exists("outcome.csv")
            outcome_df.to_csv("outcome.csv", mode='a', index=False, header=write_header)
        
        print("Simulation completed. Results saved to week_detail.csv and outcome.csv")

if __name__ == "__main__":
    main() 

Loaded 680 strategy combinations from action_df.csv
Running simulation 1/680: Combo 0
Running simulation 2/680: Combo 1
Running simulation 3/680: Combo 2
Running simulation 4/680: Combo 3
Running simulation 5/680: Combo 4
Running simulation 6/680: Combo 5
Running simulation 7/680: Combo 6
Running simulation 8/680: Combo 7
Running simulation 9/680: Combo 8
Running simulation 10/680: Combo 9
Running simulation 11/680: Combo 10
Running simulation 12/680: Combo 11
Running simulation 13/680: Combo 12
Running simulation 14/680: Combo 13
Running simulation 15/680: Combo 14
Running simulation 16/680: Combo 15
Running simulation 17/680: Combo 16
Running simulation 18/680: Combo 17
Running simulation 19/680: Combo 18
Running simulation 20/680: Combo 19
Running simulation 21/680: Combo 20
Running simulation 22/680: Combo 21
Running simulation 23/680: Combo 22
Running simulation 24/680: Combo 23
Running simulation 25/680: Combo 24
Running simulation 26/680: Combo 25
Running simulation 27/680: Comb

: 

## NO YEAR version

In [1]:
#!/usr/bin/env python3
"""
Simplified Retail Markdown Game Simulator
=========================================
This script automates playing the Retailer Game using predefined price markdown strategies.
It records results in a single CSV file:
- outcome.csv: Contains final performance metrics for each simulation
"""

import csv
import re
import pandas as pd
import itertools
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import traceback
import os

# Game URL
GAME_URL = "https://www.randhawa.us/games/retailer/nyu.html"

def generate_price_strategies():
    """
    Generate all valid price markdown strategies and returns as a DataFrame.
    
    Returns:
        pandas.DataFrame: DataFrame with all valid price combinations
    """
    # All possible prices
    prices = [60, 54, 48, 36]
    
    # Generate all valid combinations (non-increasing prices)
    valid_combos = []
    combo_id = 0
    
    for combo in itertools.product(prices, repeat=14):
        # Check if prices are non-increasing
        if all(combo[i] >= combo[i+1] for i in range(13)):
            # First week is always 60
            full_combo = [combo_id, 60] + list(combo)
            valid_combos.append(full_combo)
            combo_id += 1
    
    # Create DataFrame
    columns = ["combo_number"] + [str(i) for i in range(1, 16)]
    df = pd.DataFrame(valid_combos, columns=columns)
    
    return df

def convert_to_actions(price_df):
    """
    Convert price values to action letters.
    
    Args:
        price_df: DataFrame with price values
        
    Returns:
        pandas.DataFrame: DataFrame with action letters
    """
    action_df = pd.DataFrame()
    action_df['combo_number'] = price_df['combo_number']
    
    # First week is always maintain (A)
    action_df['1'] = 'A'
    
    # For subsequent weeks, determine the action based on price change
    for week in range(2, 16):
        curr_week = str(week)
        prev_week = str(week - 1)
        
        actions = []
        for _, row in price_df.iterrows():
            prev_price = row[prev_week]
            curr_price = row[curr_week]
            
            if curr_price == prev_price:
                action = 'A'  # Maintain
            elif curr_price == 54:
                action = 'B'  # 10% markdown
            elif curr_price == 48:
                action = 'C'  # 20% markdown
            elif curr_price == 36:
                action = 'D'  # 40% markdown
            else:
                action = '?'  # Unknown (shouldn't happen)
                
            actions.append(action)
            
        action_df[curr_week] = actions
    
    return action_df

def setup_browser():
    """
    Set up and return a configured browser instance.
    
    Returns:
        webdriver.Chrome: Configured Chrome WebDriver
    """
    chrome_options = Options()
    # Uncomment to run headless (no GUI)
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    # Performance optimizations
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-images")
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")
    chrome_options.page_load_strategy = 'eager'
    
    # Initialize the browser
    try:
        # If using webdriver_manager
        from webdriver_manager.chrome import ChromeDriverManager
        browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    except:
        # Fallback to local chromedriver
        browser = webdriver.Chrome(options=chrome_options)
    
    # Set shorter timeouts
    browser.set_page_load_timeout(5)
    browser.set_script_timeout(5)
        
    # Navigate to the game and maximize window
    browser.get(GAME_URL)
    browser.maximize_window()
    
    return browser

def run_simulation(browser, combo_row, simulation_number):
    """
    Run a single simulation using the specified pricing strategy.
    
    Args:
        browser: WebDriver instance
        combo_row: DataFrame row with actions for each week
        simulation_number: Identifier for this simulation
    
    Returns:
        list: outcome_data containing final results
    """
    wait = WebDriverWait(browser, 2)
    
    # Map letters to button IDs
    action_map = {
        'A': 'maintainButton',  # Maintain price
        'B': 'tenButton',       # 10% markdown
        'C': 'twentyButton',    # 20% markdown
        'D': 'fortyButton'      # 40% markdown
    }
    
    # Click the "Start Game" button
    try:
        start_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#practiceButton")))
        start_button.click()
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#result-table tr:nth-child(2)")))
    except Exception as e:
        # If error, refresh and try again
        print(f"Error starting game, refreshing page: {e}")
        browser.refresh()
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#practiceButton")))
        start_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#practiceButton")))
        start_button.click()
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#result-table tr:nth-child(2)")))
    
    combo_id = int(combo_row['combo_number'])
    
    # Execute the strategy for weeks 2-15 (week 1 is automatic)
    for week in range(2, 16):
        try:
            week_str = str(week)
            action_letter = combo_row[week_str]
            button_id = action_map[action_letter]
            
            # Wait for and click the appropriate button using CSS selector
            button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#{button_id}")))
            button.click()
            
        except TimeoutException:
            # Some actions might be disabled due to previous choices
            continue
        except Exception as e:
            continue
    
    # Wait for final results
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#rev")))
    
    # Extract final outcome
    try:
        # Find revenue and perfect foresight elements using CSS selectors
        revenue_elem = browser.find_element(By.CSS_SELECTOR, "#rev")
        perfect_elem = browser.find_element(By.CSS_SELECTOR, "#perfect")
        
        # Clean the text to get just the numbers
        revenue = int(re.sub(r'[^0-9]', '', revenue_elem.text))
        perfect = int(re.sub(r'[^0-9]', '', perfect_elem.text))
        
        # Extract difference percentage (from percentage element if available)
        try:
            pct_elem = browser.find_element(By.CSS_SELECTOR, "#percentage")
            difference_pct = float(re.sub(r'[^0-9.]', '', pct_elem.text))
        except:
            # Calculate manually if element not found
            difference_pct = round(((perfect - revenue) / perfect) * 100, 2) if perfect > 0 else 0
        
        outcome_data = [
            simulation_number,
            combo_id,
            revenue,
            perfect,
            difference_pct
        ]
    except Exception as e:
        print(f"Error extracting outcome: {e}")
        outcome_data = [simulation_number, combo_id, 0, 0, 0]
    
    return outcome_data

def main():
    """Main function to run the simulation"""
    # Either generate strategies or use a pre-saved CSV
    try:
        action_df = pd.read_csv('action_df.csv')
        print(f"Loaded {len(action_df)} strategy combinations from action_df.csv")
    except:
        print("Creating price strategies...")
        price_df = generate_price_strategies()
        action_df = convert_to_actions(price_df)
        action_df.to_csv('action_df.csv', index=False)
        print(f"Generated {len(action_df)} strategy combinations")
    
    # Initialize data collector
    all_outcome_data = []
    
    # Initialize browser
    browser = setup_browser()
    
    # How many simulations to run
    num_simulations = 689
    # Restart browser every X simulations
    browser_restart_frequency = 50
    
    try:
        # Loop through each combo in the action_df (limited by num_simulations)
        for index, combo_row in action_df.head(num_simulations).iterrows():
            simulation_number = index + 1
            print(f"Running simulation {simulation_number}/{num_simulations}: Combo {combo_row['combo_number']}")
            
            # Reset browser every X simulations to prevent memory issues
            if simulation_number % browser_restart_frequency == 0 and simulation_number > 0:
                print(f"Restarting browser after {simulation_number} simulations...")
                browser.quit()
                browser = setup_browser()
            
            try:
                # Run the simulation and collect results
                outcome_data = run_simulation(browser, combo_row, simulation_number)
                
                # Store results
                all_outcome_data.append(outcome_data)
                
                # Save results every 200 simulations
                if simulation_number % 200 == 0:
                    print(f"Saving interim results after {simulation_number} simulations...")
                    
                    # Save outcome data
                    outcome_df = pd.DataFrame(
                        all_outcome_data, 
                        columns=["Simulation Number", "comboID", "Your revenue", "Perfect foresight strategy", "Difference (%)"]
                    )
                    outcome_df.to_csv(f"outcome_interimSIMPLE_{simulation_number}.csv", index=False)
                
            except Exception as e:
                print(f"Error in simulation {simulation_number}: {e}")
                traceback.print_exc()
                continue
    
    except Exception as e:
        print(f"Fatal error: {e}")
        traceback.print_exc()
    
    finally:
        # Always close the browser when done
        browser.quit()
        
        # Save final results
        if all_outcome_data:
            outcome_df = pd.DataFrame(
                all_outcome_data, 
                columns=["Simulation Number", "comboID", "Your revenue", "Perfect foresight strategy", "Difference (%)"]
            )
            write_header = not os.path.exists("outcome.csv")
            outcome_df.to_csv("outcome.csv", mode='a', index=False, header=write_header)
        
        print("Simulation completed. Results saved to outcome.csv")

if __name__ == "__main__":
    main() 

Loaded 680 strategy combinations from action_df.csv
Running simulation 1/10: Combo 0
Running simulation 2/10: Combo 1
Running simulation 3/10: Combo 2
Running simulation 4/10: Combo 3
Running simulation 5/10: Combo 4
Running simulation 6/10: Combo 5
Running simulation 7/10: Combo 6
Running simulation 8/10: Combo 7
Running simulation 9/10: Combo 8
Running simulation 10/10: Combo 9
Simulation completed. Results saved to outcome.csv
